In [1]:
%matplotlib inline

In [3]:
from galsize_models.measurements import load_umachine_sdss_with_meert15
full_sdss, behroozi_complete, good_profile_mask = load_umachine_sdss_with_meert15()

sdss = full_sdss[behroozi_complete & good_profile_mask]
print("Number of M*-complete SDSS galaxies with good profile measurements = {0}".format(len(sdss)))

print(sdss.keys())

Number of M*-complete SDSS galaxies with good profile measurements = 229889
['ra', 'dec', 'z', 'bptclass', 'sm', 'sfr', 'specsfr', 'sm_fib', 'sfr_fib', 'specsfr_fib', 'withinr', 'eligiblehost', 'sm_complete', 'eligiblehost2', 'd4000', 'd4000_sub', 'd4000_n', 'd4000_n_sub', 'has_match', 'logM_mendel13', 'logMB_mendel13', 'logMD_mendel13', 'pps_mendel13', 'type_mendel13', 'deltaBD_mendel13', 'dr7_objID', 'has_meert15_match', 'r50_magr_kpc_meert15', 'Magr_tot_meert15', 'Magr_bulge_meert15', 'Magr_disk_meert15', 'gr_bulge_meert15', 'gr_disk_meert15', 'morph_type_T_meert15', 'gr_kcorr_meert15', 'bulge_to_total_rband_meert15', 'logsm_bell03_meert15', 'ssfr']


In [4]:
from slidingpercentile import python_sliding_rank

x = sdss['logsm_bell03_meert15']
y = sdss['r50_magr_kpc_meert15']
window_length = 201

sliding_rank_orders = python_sliding_rank(x, y, window_length)
size_percentile = (1. + sliding_rank_orders)/float(window_length+1)
sdss['r50_magr_kpc_meert15_percentile'] = size_percentile

In [11]:
from halotools.sim_manager import CachedHaloCatalog
halocat = CachedHaloCatalog(simname='bolplanck', redshift=0)
nhalos = len(halocat.halo_table)

from halotools.empirical_models import Moster13SmHm
model = Moster13SmHm()


In [15]:
zpeak = 1./halocat.halo_table['halo_scale_factor_mpeak'] - 1.

mstar_unity_h = model.mc_stellar_mass(prim_haloprop=halocat.halo_table['halo_mpeak'],
                                     redshift=zpeak)
mstar_mpeak = mstar_unity_h/halocat.cosmology.h/halocat.cosmology.h

from astropy.table import Table

mock = Table()

keys_from_halo_catalog = ('halo_upid', 'halo_mpeak', 'halo_scale_factor_mpeak', 
                         'halo_x', 'halo_y', 'halo_z', 
                         'halo_vx', 'halo_vy', 'halo_vz', 
                          'halo_mvir_host_halo', 'halo_spin')

mstar_mask = mstar_mpeak > 10**9
for key in keys_from_halo_catalog:
    mock[key[5:]] = halocat.halo_table[key][mstar_mask]
mock['zpeak'] = zpeak[mstar_mask]

mock['mstar_mpeak'] = mstar_mpeak[mstar_mask]

logsm_mpajhu_bins = np.linspace(9.5, 11.75, 15)
logsm_mpajhu_table = 0.5*(logsm_mpajhu_bins[:-1] + logsm_mpajhu_bins[1:])

from scipy.stats import binned_statistic

logsm_diff = sdss['logsm_bell03_meert15'] - sdss['sm']
logsm_correction_table, __, __ = binned_statistic(sdss['sm'], logsm_diff, bins=logsm_mpajhu_bins, 
                                           statistic='median')

from halotools.empirical_models import halo_mass_to_halo_radius

rvir_peak_comoving_unity_h = halo_mass_to_halo_radius(mock['mpeak'], 
                                halocat.cosmology, mock['zpeak'], 'vir')
rvir_peak_physical = rvir_peak_comoving_unity_h*mock['scale_factor_mpeak']/halocat.cosmology.h
mock['rvir_halo_zpeak'] = rvir_peak_physical*1000.

In [17]:
from galsize_models.models.bulge_disk_double_power_law import component_size_vs_rhalo

rvir_halo_kpc = mock['rvir_halo_zpeak']
normalization_all = 0.008
alpha_all = 1.
scatter_all = 0.2
size_seed=14041

mock['r50_magr_kpc_meert15'] = component_size_vs_rhalo(rvir_halo_kpc, 
                        normalization_all, alpha_all, 
                        scatter=scatter_all, seed=size_seed)

In [20]:
mask = (mock['mstar_mpeak'] > 1e10) & (mock['mstar_mpeak'] < 1.2e10)
np.count_nonzero(mask)
np.std(np.log10(mock['r50_magr_kpc_meert15'][mask]))

0.24275957650226432

In [28]:
mask = (mock['mpeak'] > 1e12) & (mock['mpeak'] < 1.2e12)

np.std(np.log10(mock['mstar_mpeak'][mask])), np.median(np.log10(mock['mstar_mpeak'][mask]))

(0.20277884938264953, 10.666828137289055)

## Check $\sigma(R_{1/2})$ reduction when using same seed for $M_{\ast}$ as for $R_{1/2}$

In [61]:
mean_mstar_unity_h = model.mean_stellar_mass(prim_haloprop=halocat.halo_table['halo_mpeak'],
                                     redshift=zpeak)
mean_mstar_mpeak = mean_mstar_unity_h/halocat.cosmology.h/halocat.cosmology.h

mean_logmstar_mpeak = np.log10(mean_mstar_mpeak)

uran = np.random.rand(len(mean_logmstar_mpeak))

from scipy.stats import norm

mc_logmstar_mpeak = norm.isf(1-uran, loc=mean_logmstar_mpeak, 
                             scale=model.param_dict[u'scatter_model_param1'])
mc_mstar_mpeak = 10**mc_logmstar_mpeak

mstar_mask2 = mc_mstar_mpeak > 10**9

mock2 = Table()

keys_from_halo_catalog = ('halo_upid', 'halo_mpeak', 'halo_scale_factor_mpeak', 
                         'halo_x', 'halo_y', 'halo_z', 
                         'halo_vx', 'halo_vy', 'halo_vz', 
                          'halo_mvir_host_halo', 'halo_spin')

mstar_mask = mstar_mpeak > 10**9
for key in keys_from_halo_catalog:
    mock2[key[5:]] = halocat.halo_table[key][mstar_mask2]

mock2['mstar_mpeak'] = mc_mstar_mpeak[mstar_mask2]
mock2['uran'] = uran[mstar_mask2]
zpeak = 1./halocat.halo_table['halo_scale_factor_mpeak'] - 1.
mock2['zpeak'] = zpeak[mstar_mask2]


In [62]:
mask = (mock2['mpeak'] > 1e12) & (mock2['mpeak'] < 1.2e12)

np.std(np.log10(mock2['mstar_mpeak'][mask])), np.median(np.log10(mock2['mstar_mpeak'][mask]))

(0.20409148593002457, 10.667291997982419)

In [71]:
from halotools.empirical_models import halo_mass_to_halo_radius

rvir_peak_comoving_unity_h = halo_mass_to_halo_radius(mock2['mpeak'], 
                                halocat.cosmology, mock2['zpeak'], 'vir')
rvir_peak_physical = rvir_peak_comoving_unity_h*mock2['scale_factor_mpeak']/halocat.cosmology.h
mock2['rvir_halo_zpeak'] = rvir_peak_physical*1000.

rvir_halo_kpc = mock2['rvir_halo_zpeak']
normalization_all = 0.008
alpha_all = 1.
scatter_all = 0.25
size_seed=14041

mean_size = normalization_all*(rvir_halo_kpc)**alpha_all
log_mean_size = np.log10(mean_size)

mc_logrhalf2_mpeak = norm.isf(1-mock2['uran'], loc=log_mean_size, 
                             scale=scatter_all)
mock2['r50_magr_kpc_meert15_uran'] = 10**mc_logrhalf2_mpeak

mock2['r50_magr_kpc_meert15'] = component_size_vs_rhalo(rvir_halo_kpc, 
                        normalization_all, alpha_all, 
                        scatter=scatter_all, seed=size_seed)

In [72]:
mask = (mock2['mstar_mpeak'] > 1e10) & (mock2['mstar_mpeak'] < 1.2e10)

print(np.std(np.log10(mock2['r50_magr_kpc_meert15'][mask])), np.std(np.log10(mock2['r50_magr_kpc_meert15_uran'][mask])))

print(np.median(np.log10(mock2['mstar_mpeak'][mask])), np.median(np.log10(mock2['mstar_mpeak'][mask])))

(0.28545019830768498, 0.23931561546780455)
(10.039381910107195, 10.039381910107195)


In [ ]:
from matplotlib import ticker 

fig, ax = plt.subplots(1, 1)

__=ax.loglog()

__=ax.scatter(10**sdss['logsm_bell03_meert15'][::20], 
              sdss['r50_magr_kpc_meert15'][::20], 
             s=1, color='gray')

__=ax.plot(10**logsm_mids, median_rhalf_all_mock, color='k', lw=2)


xmin, xmax = 10**9.5, 10**12
ymin, ymax = 0.5, 30
xlabel = r'$M_{\ast}$ $[M_{\odot}]$'
ylabel = r'$\langle R_{1/2} \rangle$ $[{\rm kpc}]$'
xlim = ax.set_xlim(xmin, xmax)
ylim = ax.set_ylim(ymin, ymax)
__=ax.get_yaxis().set_major_formatter(ticker.ScalarFormatter())
__=ax.get_yaxis().set_minor_formatter(ticker.NullFormatter())
__=ax.set_yticks((1, 3, 10, 30))

__=ax.set_ylabel(ylabel)
__=ax.set_xlabel(xlabel)

title = ax.set_title(r'all galaxies')
